# Agentic Rag and External Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
   <p> 💻 &nbsp; <b>Create/Add to</b> <code>requirements.txt</code>
    
<code>
    python_dotenv==1.0.1
    letta==0.6.50
</code>
</div>

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
                                                                                                                                    
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key
openai_api_key = get_openai_api_key()

## Section 0: Setup a Letta client

In [20]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [21]:
def print_message(message):  
    if message.message_type == "reasoning_message": 
        print("🧠 Reasoning: " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print("🤖 Agent: " + message.content) 
    elif message.message_type == "tool_call_message": 
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message": 
        print("👤 User Message: " + message.content)

## Section 1: Data Sources

### Creating a source

In [22]:
source = client.sources.create(
    name="employee_handbook",
    embedding="openai/text-embedding-3-small"
)
source

Source(id='source-c0dc4b47-34ac-437c-b8c3-544ad5f271e3', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 31, 9, 33, 33), updated_at=datetime.datetime(2025, 8, 31, 9, 33, 33), organization_id='org-00000000-0000-4000-8000-000000000000')

### Uploading a source

In [23]:
job = client.sources.files.upload(
    source_id=source.id,
    file=open("handbook.pdf", "rb")
)

In [24]:
job.status

'created'

### Viewing job status over time

In [25]:
import time
from letta_client import JobStatus

while job.status != 'completed':
    job = client.jobs.retrieve(job.id)
    print(job.status)
    time.sleep(1)

completed


### Viewing job metadata

In [26]:
job.metadata

{'type': 'embedding',
 'filename': 'handbook.pdf',
 'source_id': 'source-c0dc4b47-34ac-437c-b8c3-544ad5f271e3',
 'num_passages': 11,
 'num_documents': 1}

In [27]:
passages = client.sources.passages.list(
    source_id=source.id,
)
len(passages)

0

### Creating an agent and attaching sources

In [28]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

In [29]:
agent_state = client.agents.sources.attach(
    agent_id=agent_state.id, 
    source_id=source.id
)

### Viewing agent's attached sources

In [30]:
client.agents.sources.list(agent_id=agent_state.id)

[Source(id='source-c0dc4b47-34ac-437c-b8c3-544ad5f271e3', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 31, 9, 33, 33), updated_at=datetime.datetime(2025, 8, 31, 9, 33, 33), organization_id='org-00000000-0000-4000-8000-000000000000')]

In [31]:
passages = client.agents.passages.list(agent_id=agent_state.id)
len(passages)

0

### Messaging agents and referencing attached sources

In [32]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Search archival for our company's vacation policies"
        }
    ]
)
for message in response.messages:
    print_message(message)

🧠 Reasoning: Searching for vacation policies in archival memory. No specific details given yet.
🔧 Tool Call: archival_memory_search
{
  "query": "vacation policies",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
🔧 Tool Return: ([], 0)
🧠 Reasoning: No vacation policies found in archival memory. Need to let Sarah know.
🤖 Agent: I couldn't find any information on vacation policies in our records. Is there something specific you’d like to know about them?


## Section 2: Connecting Data with Custom Tools

### Creating a custom tool

In [33]:
def query_birthday_db(name: str):
    """
    This tool queries an external database to
    lookup the birthday of someone given their name.

    Args:
        name (str): The name to look up

    Returns:
        birthday (str): The birthday in mm-dd-yyyy format

    """
    my_fake_data = {
        "bob": "03-06-1997",
        "sarah": "07-06-1993"
    }
    name = name.lower()
    if name not in my_fake_data:
        return None
    else:
        return my_fake_data[name]

In [34]:
birthday_tool = client.tools.upsert_from_function(func=query_birthday_db)

### Creating an agent with access to tools

In [35]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a agent with access to a birthday_db " \
            + "that you use to lookup information about users' birthdays."
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small",
    tool_ids=[birthday_tool.id],
    #tool_exec_environment_variables={"DB_KEY": "my_key"}
)

In [36]:
# send a message to the agent
response = client.agents.messages.create_stream(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "whens my bday????"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: User asked about their birthday; checking the birthday database for information.
🔧 Tool Call: query_birthday_db
{
  "name": "Sarah",
  "request_heartbeat": true
}
🔧 Tool Return: 07-06-1993
🧠 Reasoning: Got the birthday information! Sending it to Sarah.
🤖 Agent: Your birthday is on July 6th, 1993! 🎉
